In [1]:
import os

import argparse
import datetime as dt
import regex as re

import yaml

import requests
import smtplib
import ssl
import mimetypes
from email.message import EmailMessage

import pandas as pd

import db_handler

In [2]:
"""Jmailer method."""

# inputs = parse_args()

# config_path = inputs.config_path
# db_identifier = inputs.db_identifier
# db_table = inputs.db_table
# sender = inputs.sender
# recipients = inputs.recipients
# recipients_path = inputs.recipients_path
# email_config_path = inputs.email_config_path
# subject = inputs.subject
# credentials_path = inputs.credentials_path
# test_mode = inputs.test_mode

# --config_path
config_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/software/jmailer/config.yaml"
# --sender
sender = "jaime.meriz13@gmail.com"
# --db_identifier
db_identifier = "1t1wGAQvZuwEWOOgcgtBaqbZoafG_ZCfTV5QGyMfYHTg"
# --db_table
db_table = "contacts"
# --recipients_path
recipients_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/email_recipients.csv"
# --body_path
body_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/Email Outreaches/generic_1/body.txt"
# --email_config_path
email_config_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/Email Outreaches/generic_1/config.yaml"
# --attachments_path
attachments_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/lopez_jaime_resume_2sep2023.pdf"
# --credentials_path
credentials_path = "/Users/jaimemerizalde/Desktop/JOBS 2023/software/jmailer/secrets/db_secret.json"
# --test_mode
test_mode = "Y"

if test_mode == "Y":
    print(f"Running in test mode. Emails will be sent to {sender}")
    test_mode = True
else:
    test_mode = False

# # body = inputs.body
# # body_path = inputs.body_path
# # email_config_path = inputs.email_config_path
# if body != None and (body_path != None or email_config_path != None):
#     print("Error: Provide body or body_path and email_config_path but not both. Defaulting to body provided.")
#     return

# attachments = inputs.attachments_path
# print("Parsed args flow complete.")

# config = yaml.safe_load(open(config_path))
# credentials = parse_config(config)
# print("Loaded credentials flow complete.")

# gmail_password = credentials["gmail"]["app_password"]
# print("Get passwords flow complete.")

# recipients = get_csv_as_list(recipients_path)
# print("Get recipients flow complete.")

# if email_config_path != None:
#     email_config = yaml.safe_load(open(email_config_path))
#     print("Load email config flow complete.")

# clearbit_api_key = credentials["clearbit"]["api_key"]
# print("Clearbit user flow complete.")

# context = ssl.create_default_context()
# smpt_connection = smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) 
# smpt_connection.login(sender, gmail_password)
# print("SMPT connection flow complete.")

# # THIS RIGHT NOW IS A PROBLEM. I can't rely on Clearbit to get the names so I have to get a workaround...
# names = Clearbit().get_names_from_email_list(recipients, username=clearbit_api_key)


SyntaxError: invalid syntax (653307715.py, line 26)